# Imputation

## import Library

In [2]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

In [3]:
m = [[1,np.nan,3],[np.nan,np.nan,6],[np.nan,9,3]]
m

[[1, nan, 3], [nan, nan, 6], [nan, 9, 3]]

In [4]:
imp_mean = SimpleImputer(missing_values= np.nan, strategy= 'mean')
imp_mean.fit(m)
df = [[np.nan, 1,np.nan],[4, 0, 6], [np.nan, 2, np.nan]]

In [5]:
p = imp_mean.transform(df)
# replace the missing data with the mean of orignal data of each columns not the current data 

# Holidays 

## Import Library

In [6]:
from workalendar.europe.united_kingdom import UnitedKingdom

ModuleNotFoundError: No module named 'workalendar'

In [ ]:
UnitedKingdom().holidays()

[(datetime.date(2024, 1, 1), 'New year'),
 (datetime.date(2024, 3, 29), 'Good Friday'),
 (datetime.date(2024, 3, 31), 'Easter Sunday'),
 (datetime.date(2024, 4, 1), 'Easter Monday'),
 (datetime.date(2024, 5, 6), 'Early May Bank Holiday'),
 (datetime.date(2024, 5, 27), 'Spring Bank Holiday'),
 (datetime.date(2024, 8, 26), 'Late Summer Bank Holiday'),
 (datetime.date(2024, 12, 25), 'Christmas Day'),
 (datetime.date(2024, 12, 26), 'Boxing Day')]

In [ ]:
from typing import List
from dateutil.relativedelta import relativedelta, TH
import datetime
from workalendar.usa import California

In [ ]:
class holidays:
  def __init__(self, year):
    self.year = year
  
  
  def create_custom_holidays(self) -> List:
    self.new_holidays = datetime.datetime(self.year, 11,1) + relativedelta(weekday= TH(+4)) +datetime.timedelta(days = 1)
    self.custom_hlidays = California().holidays()
    self.custom_hlidays.append((self.new_holidays.date(),'Black Friday'))
    # return {k : v for (k, v) in custom_hlidays}
    return self.custom_hlidays
  
  def get_holidays_date(self) -> List:
    self.holidays_list = [u for (u, v) in self.create_custom_holidays()]
    return self.holidays_list
  
  def is_holiday(self, current_date: datetime.date):
    """Determine if we have a holiday"""
    if current_date in self.get_holidays_date():
      return True
    else :
      return False
  
  

In [ ]:
today = datetime.date(2024, 1, 15)
ho = holidays(2024)
ho.create_custom_holidays()
ho.get_holidays_date()
ho.is_holiday(today)


True

# Date Annotation

In [ ]:
import calendar
calendar.monthrange(2024,1)

(0, 31)

In [ ]:
def year_anchor(current_date : datetime.date):
  return (
    (current_date - datetime.date(current_date.year, 1, 1)).days,
    (datetime.date(current_date.year, 12, 31) - current_date).days
  )

In [ ]:
now = datetime.date.today()

In [ ]:
year_anchor(now)

(79, 286)

In [ ]:
calendar.monthcalendar(now.year, now.month)[2][calendar.FRIDAY]

15

In [ ]:
def get_last_friday(current_date: datetime.date, weekday = calendar.FRIDAY):
  return max(week[weekday] for week in calendar.monthcalendar(current_date.year, current_date.month))

In [ ]:
get_last_friday(now)

29

## Season

In [ ]:
YEAR = 2021
seasons = [
  ('winter', (datetime.date(YEAR, 1, 1), datetime.date(YEAR, 3, 20))),
  ('spring', (datetime.date(YEAR, 3, 21), datetime.date(YEAR, 6, 20))),
  ('summer', (datetime.date(YEAR, 6, 21), datetime.date(YEAR, 9, 22))),
  ('autumn', (datetime.date(YEAR, 9, 23), datetime.date(YEAR, 12, 20))),
  ('winter', (datetime.date(YEAR, 12, 21), datetime.date(YEAR, 12, 31)))
]




def is_in_interval(current_date: datetime.date, seasons):
  # return next(season for (season , (start, end)) in seasons if start <= current_date.replace(year = YEAR) <= end) 
  
  
  
  
  # season_list = [season for (season , (start, end)) in seasons]
  for i,j in enumerate(seasons):
    if j[1][0] <= current_date.replace(year = YEAR) <= j[1][1] :
      return j[0]
    else:
      return False
  
    
    
    

In [ ]:
now1 = datetime.date.today()

In [ ]:
now1

datetime.date(2024, 3, 20)

In [ ]:
is_in_interval(now1, seasons)

'winter'

# Automated Feature Extraction

In [ ]:
import featuretools as ft
from featuretools.primitives import Minute, Hour, Day, Month, Year, Weekday

In [ ]:
data = pd.DataFrame(
  {'Time':['2014-01-01 01:41:50',
           '2014-01-01 02:06:50',
           '2014-01-01 02:31:50',
           '2014-01-01 02:56:50',
           '2014-01-01 03:21:50'],
   
    'Target': [0, 0, 0, 0, 1]
  }
)
data

,Time,Target
0,2014-01-01 01:41:50,0
1,2014-01-01 02:06:50,0
2,2014-01-01 02:31:50,0
3,2014-01-01 02:56:50,0
4,2014-01-01 03:21:50,1


In [ ]:
from datetime import datetime

In [ ]:
data['index'] = data.index

In [ ]:
# data.Time = data.Time.apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S'))

In [ ]:
es = ft.EntitySet('My EntitySet')
es.add_dataframe(
  dataframe = data,
  dataframe_name = 'data_name', 
  index = 'index', 
  time_index = 'Time'
)
es

Entityset: My EntitySet
  DataFrames:
    data_name [Rows: 5, Columns: 3]
  Relationships:
    No relationships

In [ ]:
fm, features = ft.dfs(
  entityset= es, 
  dataframes = data,
  target_dataframe_name = 'data_name',
  max_depth = 1,
  trans_primitives= [Minute, Hour, Day, Month, Year, Weekday])

In [ ]:
fm

,Target,DAY(Time),HOUR(Time),MINUTE(Time),MONTH(Time),WEEKDAY(Time),YEAR(Time)
index,,,,,,,
0,0,1,1,41,1,2,2014
1,0,1,2,6,1,2,2014
2,0,1,2,31,1,2,2014
3,0,1,2,56,1,2,2014
4,1,1,3,21,1,2,2014


In [7]:
import  numpy as np
import pandas as pd


In [8]:
x = np.array([[1,0], [4,8,],[8,23]])
x_covariance_matrix = np.cov(x)

In [9]:
help(np.linalg.eig)

Help on _ArrayFunctionDispatcher in module numpy.linalg:

eig(a)
    Compute the eigenvalues and right eigenvectors of a square array.
    
    Parameters
    ----------
    a : (..., M, M) array
        Matrices for which the eigenvalues and right eigenvectors will
        be computed
    
    Returns
    -------
    A namedtuple with the following attributes:
    
    eigenvalues : (..., M) array
        The eigenvalues, each repeated according to its multiplicity.
        The eigenvalues are not necessarily ordered. The resulting
        array will be of complex type, unless the imaginary part is
        zero in which case it will be cast to a real type. When `a`
        is real the resulting eigenvalues will be real (0 imaginary
        part) or occur in conjugate pairs
    
    eigenvectors : (..., M, M) array
        The normalized (unit "length") eigenvectors, such that the
        column ``eigenvectors[:,i]`` is the eigenvector corresponding to the
        eigenvalue ``eigenval

In [10]:
eig_vals, eig_vecs = np.linalg.eig(x_covariance_matrix)

In [11]:
eig_vals

array([0.00000000e+00, 1.21000000e+02, 2.10984964e-15])

In [12]:
eig_vecs

array([[-0.99793175, -0.06428243,  0.23386171],
       [-0.01656318,  0.25712974, -0.93544683],
       [-0.06211193,  0.96423652,  0.26504327]])

In [13]:
a = pd.DataFrame(eig_vecs)
a

,0,1,2
0,-0.997932,-0.064282,0.233862
1,-0.016563,0.257130,-0.935447
2,-0.062112,0.964237,0.265043


In [23]:
help(a.sum)

Help on method sum in module pandas.core.frame:

sum(axis: 'Axis | None' = 0, skipna: 'bool' = True, numeric_only: 'bool' = False, min_count: 'int' = 0, **kwargs) method of pandas.core.frame.DataFrame instance
    Return the sum of the values over the requested axis.
    
    This is equivalent to the method ``numpy.sum``.
    
    Parameters
    ----------
    axis : {index (0), columns (1)}
        Axis for the function to be applied on.
        For `Series` this parameter is unused and defaults to 0.
    
        .. warning::
    
            The behavior of DataFrame.sum with ``axis=None`` is deprecated,
            in a future version this will reduce over both axes and return a scalar
            To retain the old behavior, pass axis=0 (or do not pass axis).
    
        .. versionadded:: 2.0.0
    
    skipna : bool, default True
        Exclude NA/null values when computing the result.
    numeric_only : bool, default False
        Include only float, int, boolean columns. Not 

In [31]:
a.pow(2)

,0,1,2
0,0.995868,0.004132,0.054691
1,0.000274,0.066116,0.875061
2,0.003858,0.929752,0.070248


In [30]:
a.iloc[:,0].pow(2)

0    0.995868
1    0.000274
2    0.003858
Name: 0, dtype: float64

In [24]:
a.pow(2).sum('index')

0    1.0
1    1.0
2    1.0
dtype: float64

In [ ]:
eig_pairs = []

In [ ]:
-0.99793175 * (-0.99793175) + (-0.06428243) * (-0.06428243) +(0.2338617) *(0.2338617)

1.0546913031916576